In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal

In [2]:
path_file = "D:\Edu\Lab\Bin_Classifier_Project\data/41649_result_data_v1.csv"   # total_data_v4

df = pd.read_csv(path_file)
df.head()

,D_ID,Ch,Y,Length,Rate,0,1,2,3,4,...,502,503,504,505,506,507,508,509,510,511
0,41649,ch11,1.0,0.015900,10.0,0.047000,0.047354,0.046545,0.044573,0.041437,...,0.068431,0.068107,0.067647,0.066334,0.064166,0.061622,0.059619,0.058182,0.057308,0.057000
1,41649,ch11,0.0,0.013423,10.0,0.008656,0.012800,0.016814,0.020705,0.024493,...,0.115163,0.117910,0.120869,0.123986,0.127094,0.129613,0.131437,0.132614,0.133136,0.133000
2,41649,ch11,1.0,0.017600,10.0,0.051000,0.050911,0.050553,0.049928,0.049035,...,0.017358,0.019915,0.023375,0.026494,0.028777,0.030293,0.031837,0.033637,0.035691,0.038000
3,41649,ch11,0.0,0.016390,10.0,-0.009367,-0.006322,-0.003447,-0.000969,0.000002,...,0.030438,0.028142,0.026082,0.023952,0.022153,0.021232,0.021357,0.022458,0.024524,0.027556
4,41649,ch11,1.0,0.015381,10.0,0.074073,0.073189,0.072425,0.071784,0.070848,...,-0.056633,-0.055225,-0.052649,-0.048840,-0.043757,-0.036748,-0.027419,-0.016791,-0.005622,0.006088


In [3]:
stats_df = pd.DataFrame(np.column_stack([df.Y, df.Length]), columns=["Y", "Length"])
signal_fragments = df.loc[:, df.columns[5:]].to_numpy()
stats_df.shape

(654, 2)

In [4]:
def find_max_num(arr: np.array, arr2: np.array = None):
    f_increase = False
    max_count = 0
    max_v = 0
    maximums = []

    for i in range(1, arr.shape[0]):
        if not f_increase and arr[i - 1] < arr[i]:
            f_increase = True
            max_v = arr[i]
            # print(arr[i - 1], arr[i])
        if f_increase:
            max_v = max(arr[i], max_v)
            if arr[i - 1] > arr[i] and abs(max_v - arr[i]) / max_v > 0.3:
                f_increase = False
                # print(max_v, arr[i], abs(max_v - arr[i]) / max_v, "\n")
                maximums.append(max_v)
                max_count += 1
    if len(maximums) > 0:
        if arr2 is not None:
            max_max_freq = arr2[arr == max(maximums)][0]
        else:
            max_max_freq = -1.0
        if len(maximums) > 1:
            maximums = np.array(maximums)
            mean_max_ratio = (maximums[maximums != maximums.max()] / maximums.max()).max()
        else:
            mean_max_ratio = 1.0
    else:
        return [0.0, 0.0, -1.0]

    return [max_count, mean_max_ratio, max_max_freq]


def down_to_zero(x, edge=0.05):
    return x if x > edge else 0.0


def fragment_fft(arr: np.array):
    fft = np.fft.fft(arr)
    fft_v = fft.real ** 2 + fft.imag ** 2

    filter_values = np.vectorize(lambda x: down_to_zero(x, edge=fft_v.max() * 0.2))
    fft_v_filter = filter_values(fft_v)

    frequency = np.unique(np.abs(np.fft.fftfreq(arr.shape[0])))
    fft_v_filter = fft_v_filter[:frequency.shape[0]]

    return [frequency, fft_v_filter]

In [9]:
from scipy.stats import skew

fft_stats = []
for i in range(signal_fragments.shape[0]):
    signal_data = signal_fragments[i]
    max_point, min_point = signal_data.max(), signal_data.min()
    # normalise all values
    # signal_data = (signal_data - (max_point + min_point) / 2) / (max_point - min_point)

    n_diff = 1
    signal_data_d1 = np.diff(signal_data, n=n_diff)

    b, a = signal.butter(5, 0.1)
    signal_data_f = signal.filtfilt(b, a, signal_data_d1)

    fft = np.fft.fft(signal_data_f)
    fft_v = fft.real ** 2 + fft.imag ** 2

    filter_values = np.vectorize(lambda x: down_to_zero(x, edge=fft_v.max() * 0.05))
    fft_v_filter = filter_values(fft_v)  # fft_v

    frequency = np.unique(np.abs(np.fft.fftfreq(signal_data_f.shape[0])))
    fft_v_filter = fft_v_filter[:frequency.shape[0]]

    # print(fft_v[(frequency > 0) & (frequency < 0.1)])
    fft_stats.append(find_max_num(fft_v_filter, frequency) + [np.abs(skew(signal_data))])

fft_stats = np.array(fft_stats)
# fft_stats = np.column_stack([fft_stats, signal_fragments])
fft_stats.shape

(654, 4)

In [10]:
# def get_fft_stats(fragment: np.array):
#     fragment_d1 = np.diff(fragment, n=1)
#     b, a = signal.butter(5, 0.1)
#
#     filtered_fragment = signal.filtfilt(b, a, fragment_d1)
#     freq, fft = fragment_fft(filtered_fragment)
#     return np.array(find_max_num(freq, fft))
#
# def fragments_d1(fragment: np.array):
#     # print(fragment)
#     return np.diff(fragment, n=1)
#
# # signal_fragments_d1 = np.apply_along_axis(fragments_d1, 1, signal_fragments)
#
# fft_stats = np.apply_along_axis(get_fft_stats, 1, signal_fragments)
# fft_stats.shape

In [11]:
stats_df[["max_count", "mean_max_ratio", "max_freq", "skewness"]] = fft_stats

print(stats_df.shape)
stats_df.head()

(654, 6)


,Y,Length,max_count,mean_max_ratio,max_freq,skewness
0,1.0,0.015900,1.0,1.000000,0.015656,0.237655
1,0.0,0.013423,3.0,0.596221,0.019569,0.107958
2,1.0,0.017600,1.0,1.000000,0.015656,0.309653
3,0.0,0.016390,8.0,0.492741,0.029354,0.127061
4,1.0,0.015381,7.0,0.796791,0.019569,0.287986


In [12]:
print(stats_df.groupby(["Y", "max_count"]).agg(["count", "mean"]))  # [["mean_max_ratio", "max_freq"]]

              Length           mean_max_ratio           max_freq            \
               count      mean          count      mean    count      mean   
Y   max_count                                                                
0.0 1.0            2  0.011600              2  1.000000        2  0.009785   
    2.0           28  0.010429             28  0.322602       28  0.013629   
    3.0           54  0.013010             54  0.460877       54  0.015148   
    4.0          100  0.015561            100  0.543004      100  0.017084   
    5.0          106  0.017679            106  0.600608      106  0.019920   
    6.0           80  0.018949             80  0.691660       80  0.021233   
    7.0           64  0.022517             64  0.700724       64  0.025501   
    8.0           39  0.024395             39  0.679601       39  0.025440   
    9.0           24  0.024363             24  0.789530       24  0.025603   
    10.0           5  0.020840              5  0.780846        5

In [151]:
def plot_signal_fft(log_df, fig, axs):
    x_ = np.arange(0, 512, 1)

    for col in log_df.columns[1:]:
        y_ = log_df[col]

        fft = np.fft.fft(y_)
        fft_v = fft.real ** 2 + fft.imag ** 2

        # filter_values = np.vectorize(lambda x: down_to_zero(x, edge=fft_v.max() * 0.2))
        fft_v_filter = fft_v  # filter_values(fft_v)

        frequency = np.unique(np.abs(np.fft.fftfreq(y_.shape[0])))
        fft_v_filter = fft_v_filter[:frequency.shape[0]]
        m, r, f = find_max_num(fft_v_filter)
        axs[0].plot(frequency, fft_v_filter, label=f"{col}. C_max: {m}, Rat: {round(r, 2)}, Fr: {round(f, 2)}")
        axs[1].plot(x_, y_, label=col)

    axs[0].set_title(f'FFT', fontdict={'fontsize': 10}, loc="left")
    axs[0].set_xlabel('Freq [Hz]')
    axs[0].grid(True)
    axs[0].set_ylim([0, 25])
    axs[0].legend(loc='upper left', borderaxespad=0.)  # bbox_to_anchor=(0.005, 1),

    axs[1].set_title(f"Signal", fontdict={'fontsize': 10}, loc="left")
    axs[1].set_xlabel('Points')
    axs[1].legend(loc='upper left', borderaxespad=0.)
    # x_min = log_df["t"].min()
    # x_max = log_df["t"].max()
    # try:
    #     axs[1].set_xticks(np.round(np.arange(x_min, x_max + 1e-3, (x_max - x_min) / 8), 5))
    # except ValueError as e:
    #     pass
    axs[1].grid(True)

In [149]:
marks = stats_df["Y"].to_numpy()
max_count = stats_df["max_count"].to_numpy()

fil_ids = np.where((marks == 1) & (max_count > 2))[0]
print(fil_ids, fil_ids.shape[0])

[  5  14  22  25  26  32  38  39  44  45  47  50  52  53  57  58  59  60
  62  64  65  68  72  73  74  75  82  85  86  92  93  97  98  99 100 101
 104 105 108 112 113 119 120 122 123 126 128 129 130 131 136 149 151 157
 163 168 170 171 177 178 181 183 184 190 192 194 195 197 199 202 204 206
 210 213 214 215 216 217 222 223 224 226 229 236 237 239 240 242 243 245
 247 248 249 250 253 255 260 265 266 268 269 270 272 274 275 277 279 282
 283 287 288 290 291 292 293 294 295 296 297 299 300 302 305 307 308 309
 310 313 316 318 319 321 322 324 327 329 331 332 335 341 343 345 347 350
 351 353 355 356 358 359 360 365 369 372 374 376 378 379 381 383 386 387
 388 390 392 396 397 398 401 403 404 405 406 408 413 416 418 419 420 424
 426 427 429 431 432 433 435 436 439 441 442 447 448 449 451 454 455 456
 457 461 464 465 470 472 473 474 475 477 478 479 483 486 487 488 489 492
 494 495 497 498 499 501 502 504 505 507 509 510 511 512 513 514 515 517
 518 523 527 528 530 531 533 535 536 538 540 543 54

In [158]:
for i in range(min(fil_ids.shape[0], 10)):
    signal_data = signal_fragments[fil_ids[i]]

    n_diff = 1
    signal_data_d1 = np.diff(signal_data, n=n_diff)

    log_df = pd.DataFrame(np.column_stack([np.arange(0, 512, 1), signal_data]), columns=["t", "ch11"])
    log_df["ch11"] = signal_data
    # if log_df is not None:
    #     log_df[f"ch11_{n_diff}d"] = np.concatenate([np.diff(signal_data, n=n_diff), [0]*n_diff])

    # sos = signal.butter(5, 0.1, output="sos")
    #
    # signal_data_d1 = signal.sosfilt(sos, signal_data_d1)

    b, a = signal.butter(5, 0.1)

    signal_data_d1 = signal.filtfilt(b, a, signal_data_d1)
    log_df["ch11_b"] = np.concatenate([signal_data_d1, [0] * n_diff])

    # m = signal_data_d1.mean()
    # std = signal_data_d1.std()
    # log_df["ch11_mn"] = np.array([m] * (signal_data_d1.shape[0]+1))
    # log_df["ch11_std"] = np.array([m + std] * (signal_data_d1.shape[0]+1))
    # log_df["ch11_-std"] = np.array([m - std] * (signal_data_d1.shape[0]+1))

    fig, axs = plt.subplots(2, 1, figsize=(6, 5))
    # plt.tight_layout()
    plot_signal_fft(log_df, fig, axs)
    plt.savefig(f"img/1/41226/fragment_41226_{fil_ids[i]}.png")
    # plt.show()
    plt.close()